In [1]:
from pyspark import SparkContext, SparkConf

import pyspark

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Pyspark Cassandra") \
        .master("local[*]") \
        .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.0.1") \
        .getOrCreate()

In [3]:
df = spark.read \
  .format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("nullValue", "NA") \
  .load("/Users/deepak/Downloads/sample.csv")

In [4]:
df.show(5)

+---+---+------+
| id|age|  name|
+---+---+------+
|  1| 25|deepak|
|  2| 27|  yash|
|  3| 30|aniket|
+---+---+------+



In [5]:
df.createOrReplaceTempView("customer")

dfout = spark.sql("""select * from customer""")

In [13]:
dfout.write \
  .format("org.apache.spark.sql.cassandra") \
  .mode("overwrite") \
  .option("confirm.truncate", "true") \
  .option("spark.cassandra.connection.host", "127.0.0.1") \
  .option("spark.cassandra.connection.port", "9042") \
  .option("keyspace", "production") \
  .option("table", "test") \
  .save()

In [14]:
import cassandra
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


In [24]:
query = """select * from production.test"""

try:
    rows = session.execute(query)
    for row in rows:
        print (row)
    print("Done!")
except Exception as e:
    print(f"Table Empty Error : {e}")

Row(id=1, age=25, name='deepak')
Row(id=2, age=27, name='yash')
Row(id=3, age=30, name='aniket')
Done!
